# 31 비동기 처리

## 1. 비동기 처리의 이해
### (1) 동기 처리와 비동기 처리
- 동기 처리와 비동기 처리 비교
  - 동기 처리  (Synchronous)
  - 비동기 처리 (Asynchronous)
    - 병렬(동시)로 작업을 수행
    - 전체 실행 시간이 빠름
      - 먼저 호출한 함수의 종료와 관계없이 다음 변수 호출
   

In [1]:
# 필요한 패키지 참조

# 프로그램에 딜레이를 적용하는 기능을 제공하는 모듈
import time
# 날짜 처리 모듈
import datetime as dt
import MyMailer
#비 동기 처리 기능을 제공하는 모듈
import concurrent.futures as futures

In [4]:
# 주어진 시간 동안 딜레이가 발생하는 함수 정의
# 다소 시간이 오래 걸리는 상황 가정
    # 주어진 seconds 값 만큼 반복 수행하면서 1초씩 딜레이 시간을 갖는 함수
        # time.sleep
def timeWork(name, seconds):
    print("[%s] 작업을 %d초 동안 수행합니다."%  (name, seconds))

    for i in range (0,seconds):
        time.sleep(1)
        print("[%s] %d초..." % (name, i+1))
    
    print("[%s] 작업이 종료되었습니다." %name)


In [5]:
# 동기식 함수 호출
# 순차적으로 작업이 수행된다.
#   - 3,4,5 라인에서 호출되는 timeWork() 함수는 각각 주어진 시간동안 작업이 종료되기 전까지는 다음 라인을 진행하지 않는다.

startTime = dt.datetime.now()

timeWork("A", 3)
timeWork("B", 5)
timeWork("C", 2)

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[A] 작업을 3초 동안 수행합니다.
[A] 1초...
[A] 2초...
[A] 3초...
[A] 작업이 종료되었습니다.
[B] 작업을 5초 동안 수행합니다.
[B] 1초...
[B] 2초...
[B] 3초...
[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
[C] 작업을 2초 동안 수행합니다.
[C] 1초...
[C] 2초...
[C] 작업이 종료되었습니다.
작업에 소요된 시간은 총 10초 입니다.


In [7]:
# 비동기식 함수 호출
    # ThreadPollExecutor에 의한 비동기식 함수 호출
        # max_workers: 동시에 들어가는 작업 수 (최대 10까지)
        # .submit() 함수로 실행 
# timeWork() 함수가 작업이 종료되지 않더라도 다음 라인을 수행하기 때문에 4,5,6라인이 거의 동시에 작업을 시작한다.
# 비동기식으로 호출된 함수들은 각각 독립적으로 실행되므로 세 개의 작업은 동시성을 가짐

startTime = dt.datetime.now()

with futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.submit(timeWork, "A", 3)
    executor.submit(timeWork, "B", 5)
    executor.submit(timeWork, "C", 2)

endTime = dt.datetime.now()
workTime = endTime - startTime 
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[A] 작업을 3초 동안 수행합니다.
[B] 작업을 5초 동안 수행합니다.
[C] 작업을 2초 동안 수행합니다.
[B] 1초...[A] 1초...
[C] 1초...

[C] 2초...[A] 2초...
[B] 2초...

[C] 작업이 종료되었습니다.
[B] 3초...
[A] 3초...
[A] 작업이 종료되었습니다.
[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
작업에 소요된 시간은 총 5초 입니다.


## 2. 메일링 리스트 개선

### (1) 메일 발송에 필요한 변수 초기화
- 이전 단원의 소스코드 재사용
  

In [8]:
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "운영지원팀 <jcdasom@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding = 'utf-8') as f:
    contentTmpl = f.read()

### (2) 비동기식 메일 발송
- 동시 메일 발송 처리
  - max_workers 로 설정한 값에 따라 최대 10개 까지 동시 발송을 수행하며, 총 작업 시간이 단축된 것을 확인할 수 있다.

In [10]:
startTime = dt.datetime.now()
with open("mail/mail_list.csv", "r", encoding ="euc-kr") as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            toAddr = "{name}<{email}>".format(name = name, email = email)
            subject = subjectTmpl.format(name = name, yy = year, mm = month)
            content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)

            #MyMailer.sendMail(fromAddr, toAddr, subject, content [file1, fiel2])
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])
        
    endTime = dt.datetime.now()
    workTime = endTime - startTime
    print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

작업에 소요된 시간은 총 4초 입니다.
